In [15]:
import pandas as pd
import math
import numpy as np
import os
import glob
from shapely.geometry import Point, Polygon
import scipy.spatial

#directory where behavior videos are located
video_directory = '01_Raw_videos'
adult_directory = '02_Adult_Model'
infants_directory = '04_Infants_Model_Adjusted'
infants_raw_directory = '03_Infants_Model'
dataframe_directory = '06_Unified_dataframe'
video_info_pathway = r'09_Simba_video_info\video_info.csv'
output_directory = '08_Feature_added_dataframe'
file_location = os.path.join(video_directory, '*.mp4')

# create list of filenames for every video to be processed
# this list is the input for the video processing function
filenames = glob.glob(file_location)
videoname = []
for f in filenames:
    # establish name for output file from the input filename
    x = os.path.split(f)
    x = x[1].split('.mp4')
    x = x[0]
    videoname.append(x)

# Function to calculate the average of the infant bodypoints for each coordinate
def InfantAvg(head, middle_head, middle_tail, tail):
        series = (head + middle_head + middle_tail + tail) / 4
        return series
    
# Function for calculating euclidean distance
def EuclideanDistCald(bp1xVals, bp1yVals, bp2xVals, bp2yVals, currPixPerMM):
        series = (np.sqrt((bp1xVals - bp2xVals) ** 2 + (bp1yVals - bp2yVals) ** 2)) / currPixPerMM
        return series

# Function for calculating angle between three points
def angle3pt(ax, ay, bx, by, cx, cy):
    ang = math.degrees(
        math.atan2(cy - by, cx - bx) - math.atan2(ay - by, ax - bx))
    return ang + 360 if ang < 0 else ang

def count_values_in_range(series, values_in_range_min, values_in_range_max):
        return series.between(left=values_in_range_min, right=values_in_range_max).sum()
    
# Load in csv file with video info
video_info = pd.read_csv (video_info_pathway)
    
# Loop through all the videos
for x in videoname:
    
    # Load in the unified dataframe with points for adult, infants, and nest coordinates
    hdf_file = os.path.join(dataframe_directory, x)
    hdf_var = glob.glob(hdf_file + '*')
    Animals = pd.read_hdf(hdf_var[0])
    pd.set_option('display.max_columns', 70)
    
    # Load in the adult dataframe with points for adult, infants, and nest coordinates
    hdf_file = os.path.join(adult_directory, x)
    hdf_var = glob.glob(hdf_file + '*')
    Adult = pd.read_hdf(hdf_var[0])
    pd.set_option('display.max_columns', 1000)
    
    # Load in the infants dataframe
    hdf_file = os.path.join(infants_directory, x)
    hdf_var = glob.glob(hdf_file + '*')
    Infants = pd.read_hdf(hdf_var[0])
    
    # Load in the raw infants dataframe
    hdf_file = os.path.join(infants_raw_directory, x)
    hdf_var = glob.glob(hdf_file + '*')
    Raw_infants = pd.read_hdf(hdf_var[0])
    Raw_infants = Raw_infants.fillna(0)
    
    in_nest_joined_columns = []
    # Make a loop for the animals columns
    for animal_column in Animals.columns:
        
        # Start that will include all the in_nest lists for each of the nest columns in the dataframe
        in_nest_columns = []
        
        # Only perform operations on the x columns of adult head_center and all infant bodypoints
        if ("head_center" in animal_column  or "Infant" in animal_column[0]) and "x" in animal_column:
            
            # Loop through all the nest columns
            for nest_column in Animals.columns:
                
                # Start a list that checks whether a bodypoint is inside a nest polygon in a given frame
                # for one of the nest columns (there may be several in a dataframe)
                in_nest = []
                
                if "Nest" in nest_column:
                    
                    # Loop through all the frames of the column
                    for i in range(len(Animals)):
                        
                        # If there are no coordinates in a given frame, add a 0 to the list and skip the rest
                        if type(Animals.at[i, nest_column]) is float:
                            in_nest.append(0)
                            continue
                        
                        if "Adult" in animal_column:
                            
                            ptslist = []
                            
                            if (pd.notnull(Animals.at[i, ('Adult', 'head_center', 'x')]) and 
                               pd.notnull(Animals.at[i, ('Adult', 'head_center', 'y')])):
                                p1 = Point(Animals.at[i, ('Adult', 'head_center', 'x')], 
                                           Animals.at[i, ('Adult', 'head_center', 'y')])
                                ptslist.append(p1)
                            
                            if (pd.notnull(Animals.at[i, ('Adult', 'ear_right', 'x')]) and 
                               pd.notnull(Animals.at[i, ('Adult', 'ear_right', 'y')])):
                                p2 = Point(Animals.at[i, ('Adult', 'ear_right', 'x')], 
                                           Animals.at[i, ('Adult', 'ear_right', 'y')])
                                ptslist.append(p2)
                            
                            if (pd.notnull(Animals.at[i, ('Adult', 'side_right', 'x')]) and 
                               pd.notnull(Animals.at[i, ('Adult', 'side_right', 'y')])):
                                p3 = Point(Animals.at[i, ('Adult', 'side_right', 'x')], 
                                           Animals.at[i, ('Adult', 'side_right', 'y')])
                                ptslist.append(p3)
                            
                            if (pd.notnull(Animals.at[i, ('Adult', 'hip_right', 'x')]) and 
                               pd.notnull(Animals.at[i, ('Adult', 'hip_right', 'y')])):
                                p4 = Point(Animals.at[i, ('Adult', 'hip_right', 'x')], 
                                           Animals.at[i, ('Adult', 'hip_right', 'y')])
                                ptslist.append(p4)
                            
                            if (pd.notnull(Animals.at[i, ('Adult', 'tailbase', 'x')]) and 
                               pd.notnull(Animals.at[i, ('Adult', 'tailbase', 'y')])):
                                p5 = Point(Animals.at[i, ('Adult', 'tailbase', 'x')], 
                                           Animals.at[i, ('Adult', 'tailbase', 'y')])
                                ptslist.append(p5)
                                
                            if (pd.notnull(Animals.at[i, ('Adult', 'hip_left', 'x')]) and
                               pd.notnull(Animals.at[i, ('Adult', 'hip_left', 'y')])):
                                p6 = Point(Animals.at[i, ('Adult', 'hip_left', 'x')], 
                                           Animals.at[i, ('Adult', 'hip_left', 'y')])
                                ptslist.append(p6)
                            
                            if (pd.notnull(Animals.at[i, ('Adult', 'side_left', 'x')]) and 
                               pd.notnull(Animals.at[i, ('Adult', 'side_left', 'y')])):
                                p7 = Point(Animals.at[i, ('Adult', 'side_left', 'x')], 
                                           Animals.at[i, ('Adult', 'side_left', 'y')])
                                ptslist.append(p7)
                            
                            if (pd.notnull(Animals.at[i, ('Adult', 'ear_left', 'x')]) and 
                               pd.notnull(Animals.at[i, ('Adult', 'ear_left', 'y')])):
                                p8 = Point(Animals.at[i, ('Adult', 'ear_left', 'x')], 
                                           Animals.at[i, ('Adult', 'ear_left', 'y')])
                                ptslist.append(p8)
    
                            poly = Polygon(ptslist)
                    
                            polynest = Polygon(Animals.at[i, nest_column])

                            if not poly.is_valid or not polynest.is_valid:
                                in_nest.append(0)
                            
                            elif poly.intersects(polynest) and ((poly.area / 2) <= poly.intersection(polynest).area):
                                in_nest.append(1)
                            else:
                                in_nest.append(0)
                        
                        else:
                            # If the x or y coordinates for a certain bodypoint is null in a given frame
                            # add a 0 to the list and skip the rest
                            if pd.isnull(Animals.at[i, animal_column]) or pd.isnull(
                                Animals.at[i, (animal_column[0], animal_column[1], "y")]):
                                in_nest.append(0)
                                continue

                            # Make a shapely point from the x and y coordinates of an animal body part
                            p1 = Point(Animals.at[i, animal_column], 
                                       Animals.at[i, (animal_column[0], animal_column[1], "y")])

                            # Make a polygon from the nest coordinates in that same frame 
                            poly = Polygon(Animals.at[i, nest_column])

                            # Add a 1 to the list if the point is within the nest polygon, and a 0 if not
                            if poly.contains(p1):
                                in_nest.append(1)
                            else:
                                in_nest.append(0)
                    
                    # Append the in_nest list for that nest column to the in_nest_columns list
                    in_nest_columns.append(in_nest)

            # Start a new list for that will only include a 0 or 1 per frame, telling if the body point of an animal
            # is inside or outside of any of the nests in that particular frame
            in_nest_joined = []

            # Loop through in_nest_columns
            for i in range(len(in_nest_columns)):
                
                # Loop through each element of each list of in_nest_columns
                for j in range(len(in_nest_columns[i])):
                    
                    # In the first iteration, just add all of the 0's or 1's from that list to in_nest_list
                    if i == 0:
                        in_nest_joined.append(in_nest_columns[i][j])
                    
                    # In subsequent iterations, compare the elements in that list of in_nest_columns to the in_nest_joined
                    # element from the previous iteration
                    # If the element in in_nest_joined is a 0 and the element from in_nest_columns is a 1, change the
                    # 0 to the 1 in in_nest_joined
                    else:
                        if in_nest_columns[i][j] > in_nest_joined[j]:
                            in_nest_joined[j] = in_nest_columns[i][j]

            # Make a new column in the dataframe that specifies the animal and bodypoint being tested,
            # and make the information in that column the data from in_nest_joined
            if "Adult" in animal_column:
                Animals[('In_Nest', animal_column[0], 'body')] = in_nest_joined
                in_nest_joined_columns.append(in_nest_joined)
            else:
                Animals[('In_Nest', animal_column[0], animal_column[1])] = in_nest_joined
                in_nest_joined_columns.append(in_nest_joined)
                
######### ADULT BODY POINTS AND BODY AREA ##########
    
    columns_name = []
    
    currPixPerMM = video_info.at [videoname.index(x), 'pixels/mm']
    print(currPixPerMM)
    # Change the column names to be contained at a single level
    for column in Adult.columns:
            
        if "likelihood" in column:
            columns_name.append(column[1] + '_p')
        else:
            columns_name.append(column[1] + '_' + column[2])
    
    # Insert a column at the start marking the frame number
    frames = []
    for i in range(len(Adult)):
        frames.append(i)
        
    Adult.columns = columns_name 
    
    Adult.insert(0, '', frames)
    
    frame_adult_area = []
    
    # use for low probability detections
    
    adult_original = Adult
    
    # Calculate the area of the adult, skipping whichever bodypoint is null in the calculation
    for i in range(len(Animals)):
        ptslist = []

        if (pd.notnull(Animals.at[i, ('Adult', 'head_center', 'x')]) and 
            pd.notnull(Animals.at[i, ('Adult', 'head_center', 'y')])):
            p1 = Point(Animals.at[i, ('Adult', 'head_center', 'x')], 
                       Animals.at[i, ('Adult', 'head_center', 'y')])
            ptslist.append(p1)

        if (pd.notnull(Animals.at[i, ('Adult', 'ear_right', 'x')]) and 
            pd.notnull(Animals.at[i, ('Adult', 'ear_right', 'y')])):
            p2 = Point(Animals.at[i, ('Adult', 'ear_right', 'x')], 
                       Animals.at[i, ('Adult', 'ear_right', 'y')])
            ptslist.append(p2)

        if (pd.notnull(Animals.at[i, ('Adult', 'side_right', 'x')]) and 
            pd.notnull(Animals.at[i, ('Adult', 'side_right', 'y')])):
            p3 = Point(Animals.at[i, ('Adult', 'side_right', 'x')], 
                       Animals.at[i, ('Adult', 'side_right', 'y')])
            ptslist.append(p3)

        if (pd.notnull(Animals.at[i, ('Adult', 'hip_right', 'x')]) and 
            pd.notnull(Animals.at[i, ('Adult', 'hip_right', 'y')])):
            p4 = Point(Animals.at[i, ('Adult', 'hip_right', 'x')], 
                       Animals.at[i, ('Adult', 'hip_right', 'y')])
            ptslist.append(p4)

        if (pd.notnull(Animals.at[i, ('Adult', 'tailbase', 'x')]) and 
            pd.notnull(Animals.at[i, ('Adult', 'tailbase', 'y')])):
            p5 = Point(Animals.at[i, ('Adult', 'tailbase', 'x')], 
                       Animals.at[i, ('Adult', 'tailbase', 'y')])
            ptslist.append(p5)

        if (pd.notnull(Animals.at[i, ('Adult', 'hip_left', 'x')]) and
            pd.notnull(Animals.at[i, ('Adult', 'hip_left', 'y')])):
            p6 = Point(Animals.at[i, ('Adult', 'hip_left', 'x')], 
                       Animals.at[i, ('Adult', 'hip_left', 'y')])
            ptslist.append(p6)

        if (pd.notnull(Animals.at[i, ('Adult', 'side_left', 'x')]) and 
            pd.notnull(Animals.at[i, ('Adult', 'side_left', 'y')])):
            p7 = Point(Animals.at[i, ('Adult', 'side_left', 'x')], 
                       Animals.at[i, ('Adult', 'side_left', 'y')])
            ptslist.append(p7)

        if (pd.notnull(Animals.at[i, ('Adult', 'ear_left', 'x')]) and 
            pd.notnull(Animals.at[i, ('Adult', 'ear_left', 'y')])):
            p8 = Point(Animals.at[i, ('Adult', 'ear_left', 'x')], 
                       Animals.at[i, ('Adult', 'ear_left', 'y')])
            ptslist.append(p8)

        poly = Polygon(ptslist)
            
        frame_adult_area.append(poly.area)
        
    Adult['adult_area'] = np.array(frame_adult_area) / (currPixPerMM ** 2)
    
########## INFANT BODY POINTS AND AVERAGE BODY POINT ##########
    
    # Make list of number of infants
    colsPerInfant = 12
    infants_num = int(len(Infants.columns) / colsPerInfant)
    infants_list = []
    for i in range(infants_num):
        infants_list.append('Infant' + str(i + 1))
    
    # Reassign the column names as with the adult column names
    columns_name = []
    for column in Infants.columns:
            
        if "likelihood" in column:
            columns_name.append(column[0] + '_' + column[1] + '_p')
        else:
            columns_name.append(column[0] + '_' + column[1] + '_' + column[2])
        
    Infants.columns = columns_name
    
    # Calculate the average for both infants for the x and y coordinates
    for i in infants_list:
        
        Infants[i[:3] + i[-1] + '_points_avg_x'] = InfantAvg(Infants[i + '_head_x'].values, Infants[i + '_middle_head_x'].values,
                                                 Infants[i + '_middle_tail_x'].values, Infants[i + '_tail_x'].values)
        
      
        Infants[i[:3] + i[-1] + '_points_avg_y'] = InfantAvg(Infants[i + '_head_x'].values, Infants[i + '_middle_head_y'].values,
                                                 Infants[i + '_middle_tail_y'].values, Infants[i + '_tail_y'].values)
    
    # Join the adult and infant dataframes under the Adult name
    Adult = pd.concat([Adult, Infants], axis=1, join='inner')
    
########## NEST SPECIFIC FEATURES ##########
    
    # Find values for nest_related metrics for each frame
    frame_nest_counts = []
    frame_area_totals = []
    frame_area_avg = []
    frame_area_home = []
    frame_centroid_home_x = []
    frame_centroid_home_y = []
    frame_circularity_home = []
    for i in range(len(Animals)):
        nest_count = 0
        area_total = 0
        area_avg = 0
        area_home = 0
        centroid_home_x = 0
        centroid_home_y = 0
        circularity_home = 0
        for column in Animals.columns:
            if column[0] == 'Nest' and not type(Animals.at[i, column]) is float:
                nest_count += 1
                area_total += Polygon(Animals.at[i, column]).area
                area_avg = area_total / nest_count
                if Polygon(Animals.at[i, column]).area > area_home:
                    area_home = Polygon(Animals.at[i, column]).area
                    centroid_home_x = Polygon(Animals.at[i, column]).centroid.x
                    centroid_home_y = Polygon(Animals.at[i, column]).centroid.y
                    circularity_home = (4 * np.pi) * area_home / (Polygon(Animals.at[i, column]).length) ** 2
        
        # Append to dataframe-long lists, then assign new Adult columns to these lists
        frame_nest_counts.append(nest_count)
        frame_area_totals.append(area_total)
        frame_area_avg.append(area_avg)
        frame_area_home.append(area_home)
        frame_centroid_home_x.append(centroid_home_x)
        frame_centroid_home_y.append(centroid_home_y)
        frame_circularity_home.append(circularity_home)

    Adult['nest_centroid_home_x'] = np.array(frame_centroid_home_x)
    Adult['nest_centroid_home_y'] = np.array(frame_centroid_home_y)
    Adult['nest_area_totals'] = np.array(frame_area_totals) / (currPixPerMM ** 2)
    Adult['nest_area_home'] = np.array(frame_area_home) / (currPixPerMM ** 2)
    
    Adult_shifted = Adult.shift(periods=1)
    Adult_shifted = Adult_shifted.fillna(0)
    
    Adult['nest_counts'] = np.array(frame_nest_counts)
    Adult['nest_area_avg'] = np.array(frame_area_avg) / (currPixPerMM ** 2)
    Adult['nest_circularity_home'] = np.array(frame_circularity_home)
    
########## EUCLIDEAN DISTANCE AND MOVEMENT OF BODY POINTS PER FRAME ##########
    
    # Find the distance between each two bodypoints on the adult and between all the body points
    # and the infant averages, without repeats
    avoid_repeats = []
    for i in Adult.columns:
        for j in Adult.columns:
            if ('_x' in i and '_x' in j and 
            not 'Infant' in i and not 'Infant' in j
            and not i == j):
                if (j[:-2] + '_to_' + i[:-2]) in avoid_repeats:
                    continue
                avoid_repeats.append(i[:-2] + '_to_' + j[:-2])
                
                Adult[avoid_repeats[-1]] = EuclideanDistCald(Adult[i].values, Adult[i[:-1] + 'y'].values,
                                         Adult[j].values, Adult[j[:-1] + 'y'].values,
                                                     currPixPerMM)
    
    # Find the movement per frame of each adult bodypoint and the adult area change per frame
    for i in Adult.columns:
        if ('_x' in i or 'points_avg_x' in i 
            or 'nest_centroid_home_x' in i) and 'Infant' not in i:
            Adult['movement_' + i[:-2]] = EuclideanDistCald(Adult[i].values, Adult[i[:-1] + 'y'].values,
                                                           Adult_shifted[i].values, 
                                                           Adult_shifted[i[:-1] + 'y'].values,
                                                           currPixPerMM)
        if 'area' in i:
            Adult['area_change'] = pd.eval("Adult_shifted.adult_area - Adult.adult_area")
            Adult['total_nest_area_change'] = pd.eval("Adult_shifted.nest_area_totals - Adult.nest_area_totals")
            Adult['nest_area_home_change'] = pd.eval("Adult_shifted.nest_area_home - Adult.nest_area_home")
            Adult['nest_area_totals_change'] = pd.eval("Adult_shifted.nest_area_totals - Adult.nest_area_totals")

########## COLLAPSED MEASURES ##########
    
    Adult['Total_movement_all_bodyparts_adult'] = pd.Series(np.zeros(len(Adult)))
    for column in Adult.columns:
        if 'movement' in column and not 'Total' in column and not 'Inf' in column and not 'nest' in column:
            Adult['Total_movement_all_bodyparts_adult'] += Adult[column]
            
    Adult['Total_movement_infants'] = pd.Series(np.zeros(len(Adult)))
    for i in infants_list:
        Adult['Total_movement_infants'] += Adult['movement_' + i[:3] + i[-1] + '_points_avg']
        
########## ANGLES ##########
    
    # Adult body angle
    Adult['adult_angles'] = Adult.apply(
            lambda x: angle3pt(x['nose_x'], x['nose_y'], x['body_center_x'], x['body_center_y'], x['tailbase_x'],
                               x['tailbase_y']), axis=1)
    
    # Angle between adult head and each infant
    for i in infants_list:
        
        Adult['adult_infant' + i[-1] + '_angle'] = Adult.apply(
                lambda x: angle3pt(x['neck_x'], x['neck_y'], x['head_center_x'], x['head_center_y'],
                                   x['Inf' + i[-1] + '_points_avg_x'], x['Inf' + i[-1] + '_points_avg_y']), axis=1)
    
    # Angle between adult head and nest centroid
    
    Adult['adult_nest_home_angle'] = Adult.apply(
            lambda x: angle3pt(x['neck_x'], x['neck_y'], x['head_center_x'], x['head_center_y'],
                               x['nest_centroid_home_x'], x['nest_centroid_home_y']), axis=1)
    
########## IN NEST ##########
    
    # Assign the the in_nest values calculated earlier for the adult and for each of the body points
    i = 0
    for animal_column in Animals.columns:
        
        # Only perform operations on the x columns of adult head_center and all infant bodypoints
        if ("head_center" in animal_column  or "Infant" in animal_column[0]) and "x" in animal_column:
            
            if "Adult" in animal_column:
                Adult['In_Nest_' + animal_column[0] + '_body'] = in_nest_joined_columns[i]
                i += 1
            else:
                Adult['In_Nest_' + animal_column[0] + '_' + animal_column[1]] = in_nest_joined_columns[i]
                i += 1

########## HULL - EUCLIDIAN DISTANCES ##########
    
    Adult_hull_large_euclidean_list = []
    Adult_hull_small_euclidean_list = []
    Adult_hull_mean_euclidean_list = []
    Adult_hull_sum_euclidean_list = []
    
    for index, row in Adult.iterrows():
        
        # Make an array for each point in the adult's body for each frame
        Adult_np_array = np.array(     
            [[row['ear_left_x'], row["ear_left_y"]], [row['ear_right_x'], row["ear_right_y"]],
             [row['nose_x'], row["nose_y"]], [row['head_center_x'], row["head_center_y"]],
             [row['neck_x'], row["neck_y"]], [row['body_center_x'], row["body_center_y"]],
             [row['side_left_x'], row["side_left_y"]], [row['side_right_x'], row["side_right_y"]],
             [row['hip_left_x'], row["hip_left_y"]], [row['hip_right_x'], row["hip_right_y"]],
             [row['tailbase_x'], row["tailbase_y"]]]).astype(int)
        
        # Find ditance between each point in the adult's body in each frame
        Adult_dist_euclidean = scipy.spatial.distance.cdist(Adult_np_array, Adult_np_array, metric='euclidean')
        Adult_dist_euclidean = Adult_dist_euclidean[Adult_dist_euclidean != 0]
        
        # Find the largest, smallest, mean and sum of these distances
        Adult_hull_large_euclidean = np.amax(Adult_dist_euclidean)
        Adult_hull_small_euclidean = np.min(Adult_dist_euclidean)
        Adult_hull_mean_euclidean = np.mean(Adult_dist_euclidean)
        Adult_hull_sum_euclidean = np.sum(Adult_dist_euclidean)
        
        # Append these to lists, then make a column for each
        Adult_hull_large_euclidean_list.append(Adult_hull_large_euclidean)
        Adult_hull_small_euclidean_list.append(Adult_hull_small_euclidean)
        Adult_hull_mean_euclidean_list.append(Adult_hull_mean_euclidean)
        Adult_hull_sum_euclidean_list.append(Adult_hull_sum_euclidean)
        
    Adult['adult_largest_euclidean_distance_hull'] = list(
        map(lambda x: x / currPixPerMM, Adult_hull_large_euclidean_list))
    Adult['adult_smallest_euclidean_distance_hull'] = list(
        map(lambda x: x / currPixPerMM, Adult_hull_small_euclidean_list))
    Adult['adult_mean_euclidean_distance_hull'] = list(
        map(lambda x: x / currPixPerMM, Adult_hull_mean_euclidean_list))
    Adult['adult_sum_euclidean_distance_hull'] = list(
        map(lambda x: x / currPixPerMM, Adult_hull_sum_euclidean_list))

########## CALC ROLLING WINDOWS MEDIANS AND MEANS ##########
    
    # Rolling window prep
    roll_windows, loopy = [], 0
    roll_windows_values = [2, 5, 6, 7.5, 15]

    fps = float(video_info.at[videoname.index(x), 'fps'])
    for i in range(len(roll_windows_values)):
        roll_windows.append(int(fps / roll_windows_values[i]))
    loopy += 1
    
    # Rolling windows for adult movement features and hull distance features
    
    for column in Adult.columns:
        if ('movement' in column or 'change' in column or 'hull' in column
            or 'angle' in column or 'In_nest' in column):
            for i in range(len(roll_windows_values)):
                currentColName = column + '_median_' + str(roll_windows_values[i])
                Adult[currentColName] = Adult[column].rolling(roll_windows[i],
                                                                min_periods=1).median()
                currentColName = column + '_mean_' + str(roll_windows_values[i])
                Adult[currentColName] = Adult[column].rolling(roll_windows[i],
                                                                min_periods=1).mean()
                currentColName = column + '_sum_' + str(roll_windows_values[i])
                Adult[currentColName] = Adult[column].rolling(roll_windows[i],
                                                                min_periods=1).sum()
    
    # Rolling windows for select adult euclidean distance features
        
    for i in range(len(roll_windows_values)):
                currentColName = 'side_left_to_side_right' + '_median_' + str(roll_windows_values[i])
                Adult[currentColName] = Adult['side_left_to_side_right'].rolling(roll_windows[i],
                                                                min_periods=1).median()
                currentColName = 'side_left_to_side_right' + '_mean_' + str(roll_windows_values[i])
                Adult[currentColName] = Adult['side_left_to_side_right'].rolling(roll_windows[i],
                                                                min_periods=1).mean()
                currentColName = 'side_left_to_side_right' + '_sum_' + str(roll_windows_values[i])
                Adult[currentColName] = Adult['side_left_to_side_right'].rolling(roll_windows[i],
                                                                min_periods=1).sum()
                            
    for i in range(len(roll_windows_values)):
                currentColName = 'head_center_to_tailbase' + '_median_' + str(roll_windows_values[i])
                Adult[currentColName] = Adult['head_center_to_tailbase'].rolling(roll_windows[i],
                                                                min_periods=1).median()
                currentColName = 'head_center_to_tailbase' + '_mean_' + str(roll_windows_values[i])
                Adult[currentColName] = Adult['head_center_to_tailbase'].rolling(roll_windows[i],
                                                                min_periods=1).mean()
                currentColName = 'head_center_to_tailbase' + '_sum_' + str(roll_windows_values[i])
                Adult[currentColName] = Adult['head_center_to_tailbase'].rolling(roll_windows[i],
                                                                min_periods=1).sum()
    
    # Rolling windows for euclidean distance features between adults and infants
    
    for j in infants_list:
        for i in range(len(roll_windows_values)):
                    currentColName = ('neck_to_' + j[:3] + j[-1] + '_points_avg' + '_median_' + 
                                      str(roll_windows_values[i]))
                    Adult[currentColName] = Adult['neck_to_' + j[:3] + j[-1] + '_points_avg'].rolling(roll_windows[i],
                                                                    min_periods=1).median()
                    currentColName = ('neck_to_' + j[:3] + j[-1] + '_points_avg' + '_mean_' + 
                                      str(roll_windows_values[i]))
                    Adult[currentColName] = Adult['neck_to_' + j[:3] + j[-1] + '_points_avg'].rolling(roll_windows[i],
                                                                    min_periods=1).mean()
                    currentColName = ('neck_to_' + j[:3] + j[-1] + '_points_avg' + '_sum_' + 
                                      str(roll_windows_values[i]))
                    Adult[currentColName] = Adult['neck_to_' + j[:3] + j[-1] + '_points_avg'].rolling(roll_windows[i],
                                                                    min_periods=1).sum()

        for i in range(len(roll_windows_values)):
                    currentColName = ('body_center_to_' + j[:3] + j[-1] + '_points_avg' + '_median_' + 
                                      str(roll_windows_values[i]))
                    Adult[currentColName] = Adult['body_center_to_' + j[:3] + j[-1] + 
                                                  '_points_avg'].rolling(roll_windows[i],min_periods=1).median()
                    currentColName = ('body_center_to_' + j[:3] + j[-1] + '_points_avg' + '_mean_' + 
                                      str(roll_windows_values[i]))
                    Adult[currentColName] = Adult['body_center_to_' + j[:3] + j[-1] + 
                                                  '_points_avg'].rolling(roll_windows[i],min_periods=1).mean()
                    currentColName = ('body_center_to_' + j[:3] + j[-1] + '_points_avg' + '_sum_' + 
                                      str(roll_windows_values[i]))
                    Adult[currentColName] = Adult['body_center_to_' + j[:3] + j[-1] + 
                                                  '_points_avg'].rolling(roll_windows[i], min_periods=1).sum()

        for i in range(len(roll_windows_values)):
                    currentColName = ('tailbase_to_' + j[:3] + j[-1] + '_points_avg' + '_median_' + 
                                      str(roll_windows_values[i]))
                    Adult[currentColName] = Adult['tailbase_to_' + j[:3] + j[-1] + 
                                                  '_points_avg'].rolling(roll_windows[i], min_periods=1).median()
                    currentColName = ('tailbase_to_' + j[:3] + j[-1] + '_points_avg' + '_mean_' + 
                                      str(roll_windows_values[i]))
                    Adult[currentColName] = Adult['tailbase_to_' + j[:3] + j[-1] + 
                                                  '_points_avg'].rolling(roll_windows[i], min_periods=1).mean()
                    currentColName = ('tailbase_to_' + j[:3] + j[-1] + '_points_avg' + '_sum_' + 
                                      str(roll_windows_values[i]))
                    Adult[currentColName] = Adult['tailbase_to_' + j[:3] + j[-1] + 
                                                  '_points_avg'].rolling(roll_windows[i], min_periods=1).sum()
    
    # Rolling windows for euclidean distance features between adult and nest
                
    for i in range(len(roll_windows_values)):
                currentColName = 'neck_to_nest_centroid_home' + '_median_' + str(roll_windows_values[i])
                Adult[currentColName] = Adult['neck_to_nest_centroid_home'].rolling(roll_windows[i],
                                                                min_periods=1).median()
                currentColName = 'neck_to_nest_centroid_home' + '_mean_' + str(roll_windows_values[i])
                Adult[currentColName] = Adult['neck_to_nest_centroid_home'].rolling(roll_windows[i],
                                                                min_periods=1).mean()
                currentColName = 'neck_to_nest_centroid_home' + '_sum_' + str(roll_windows_values[i])
                Adult[currentColName] = Adult['neck_to_nest_centroid_home'].rolling(roll_windows[i],
                                                                min_periods=1).sum()
                
    for i in range(len(roll_windows_values)):
                currentColName = 'body_center_to_nest_centroid_home' + '_median_' + str(roll_windows_values[i])
                Adult[currentColName] = Adult['body_center_to_nest_centroid_home'].rolling(roll_windows[i],
                                                                min_periods=1).median()
                currentColName = 'body_center_to_nest_centroid_home' + '_mean_' + str(roll_windows_values[i])
                Adult[currentColName] = Adult['body_center_to_nest_centroid_home'].rolling(roll_windows[i],
                                                                min_periods=1).mean()
                currentColName = 'body_center_to_nest_centroid_home' + '_sum_' + str(roll_windows_values[i])
                Adult[currentColName] = Adult['body_center_to_nest_centroid_home'].rolling(roll_windows[i],
                                                                min_periods=1).sum()
                
    for i in range(len(roll_windows_values)):
                currentColName = 'tailbase_to_nest_centroid_home' + '_median_' + str(roll_windows_values[i])
                Adult[currentColName] = Adult['tailbase_to_nest_centroid_home'].rolling(roll_windows[i],
                                                                min_periods=1).median()
                currentColName = 'tailbase_to_nest_centroid_home' + '_mean_' + str(roll_windows_values[i])
                Adult[currentColName] = Adult['tailbase_to_nest_centroid_home'].rolling(roll_windows[i],
                                                                min_periods=1).mean()
                currentColName = 'tailbase_to_nest_centroid_home' + '_sum_' + str(roll_windows_values[i])
                Adult[currentColName] = Adult['tailbase_to_nest_centroid_home'].rolling(roll_windows[i],
                                                                min_periods=1).sum()
    
    # Rolling windows for euclidean distance feature between infants
    
    for j in infants_list:
        for k in infants_list:
            if (j[:3] + j[-1] + '_points_avg_to_' + k[:3] + k[-1] + '_points_avg') in Adult.columns:
                for i in range(len(roll_windows_values)):
                    currentColName = (j[:3] + j[-1] + '_points_avg_to_' + k[:3] + k[-1] + '_points_avg' + '_median_' + 
                                      str(roll_windows_values[i]))
                    Adult[currentColName] = Adult[j[:3] + j[-1] + '_points_avg_to_' + k[:3] + k[-1] + '_points_avg'
                                                 ].rolling(roll_windows[i], min_periods=1).median()
                    currentColName = (j[:3] + j[-1] + '_points_avg_to_' + k[:3] + k[-1] + '_points_avg' + '_mean_' + 
                                      str(roll_windows_values[i]))
                    Adult[currentColName] = Adult[j[:3] + j[-1] + '_points_avg_to_' + k[:3] + k[-1] + '_points_avg'
                                                 ].rolling(roll_windows[i], min_periods=1).mean()
                    currentColName = (j[:3] + j[-1] + '_points_avg_to_' + k[:3] + k[-1] + '_points_avg' + '_sum_' + 
                                      str(roll_windows_values[i]))
                    Adult[currentColName] = Adult[j[:3] + j[-1] + '_points_avg_to_' + k[:3] + k[-1] + '_points_avg'
                                                 ].rolling(roll_windows[i], min_periods=1).sum()
            
    
    # Rolling windows for euclidean distance features between infants and nest
    
    for j in infants_list: 
        for i in range(len(roll_windows_values)):
                    currentColName = (j[:3] + j[-1] + '_points_avg_to_nest_centroid_home' + '_median_' + 
                                      str(roll_windows_values[i]))
                    Adult[currentColName] = Adult[j[:3] + j[-1] + '_points_avg_to_nest_centroid_home'
                                                 ].rolling(roll_windows[i], min_periods=1).median()
                    currentColName = (j[:3] + j[-1] + '_points_avg_to_nest_centroid_home' + '_mean_' + 
                                      str(roll_windows_values[i]))
                    Adult[currentColName] = Adult[j[:3] + j[-1] + '_points_avg_to_nest_centroid_home'
                                                 ].rolling(roll_windows[i], min_periods=1).mean()
                    currentColName = (j[:3] + j[-1] + '_points_avg_to_nest_centroid_home' + '_sum_' + 
                                      str(roll_windows_values[i]))
                    Adult[currentColName] = Adult[j[:3] + j[-1] + '_points_avg_to_nest_centroid_home'
                                                 ].rolling(roll_windows[i], min_periods=1).sum()
                
########## BODY PARTS RELATIVE TO EACH OTHER ##########
    
    Adult['Tail_tip_relative_to_tailbase_body_center_nose'] = Adult['movement_tail_tip'] - (
                Adult['movement_tailbase'] + Adult['movement_body_center'] + Adult[
            'movement_nose'])
    for i in range(len(roll_windows_values)):
        currentColName = 'tail_tip_relative_to_tailbase_body_center_nose' + str(roll_windows_values[i])
        tail_end_col_name = 'movement_tail_tip_mean_' + str(roll_windows_values[i])
        tail_base_col_name = 'movement_tailbase_mean_' + str(roll_windows_values[i])
        centroid_col_name = 'movement_body_center_mean_' + str(roll_windows_values[i])
        nose_col_name = 'movement_nose_mean_' + str(roll_windows_values[i])
        
        Adult[currentColName] = Adult[tail_end_col_name] - (
                    Adult[tail_base_col_name] + Adult[centroid_col_name] + Adult[nose_col_name])
        
########## DEVIATIONS ##########
    
    deviation_columns = []
    
    # Deviations for movements
    
    Adult['Total_movement_all_bodyparts_adult_deviation'] = (Adult[
        'Total_movement_all_bodyparts_adult'].mean() - Adult['Total_movement_all_bodyparts_adult'])
    deviation_columns.append('Total_movement_all_bodyparts_adult')
    
    Adult['movement_body_center_deviation'] = (Adult[
        'movement_body_center'].mean() - Adult['movement_body_center'])
    deviation_columns.append('movement_body_center')
    
    Adult['Total_movement_infants_deviation'] = (Adult[
        'Total_movement_infants'].mean() - Adult['Total_movement_infants'])
    deviation_columns.append('Total_movement_infants')
    
    Adult['movement_nest_centroid_home_deviation'] = (Adult[
        'movement_nest_centroid_home'].mean() - Adult['movement_nest_centroid_home'])
    deviation_columns.append('movement_nest_centroid_home')
    
    # Deviations for hull distances
    
    Adult['adult_largest_euclidean_distance_hull_deviation'] = (Adult[
        'adult_largest_euclidean_distance_hull'].mean() - Adult['adult_largest_euclidean_distance_hull'])
    deviation_columns.append('adult_largest_euclidean_distance_hull')
    
    Adult['adult_smallest_euclidean_distance_hull_deviation'] = (Adult[
        'adult_smallest_euclidean_distance_hull'].mean() - Adult['adult_smallest_euclidean_distance_hull'])
    deviation_columns.append('adult_smallest_euclidean_distance_hull')
    
    Adult['adult_mean_euclidean_distance_hull_deviation'] = (Adult[
        'adult_mean_euclidean_distance_hull'].mean() - Adult['adult_mean_euclidean_distance_hull'])
    deviation_columns.append('adult_mean_euclidean_distance_hull')
    
    Adult['adult_sum_euclidean_distance_hull_deviation'] = (Adult[
        'adult_sum_euclidean_distance_hull'].mean() - Adult['adult_sum_euclidean_distance_hull'])
    deviation_columns.append('adult_sum_euclidean_distance_hull')
    
    # Deviations for area
    
    Adult['adult_area_deviation'] = (Adult['adult_area'].mean() - Adult['adult_area'])
    
    Adult['nest_area_home_deviation'] = (Adult['nest_area_home'].mean() - Adult['nest_area_home'])
    
    Adult['nest_area_totals_deviation'] = (Adult['nest_area_totals'].mean() - Adult['nest_area_totals'])
    
    # Deviations for adult-infant distances
    
    for j in infants_list:
        Adult['neck_to_' + j[:3] + j[-1] + '_points_avg_deviation'] = (Adult[
            'neck_to_' + j[:3] + j[-1] + '_points_avg'].mean() - Adult[
            'neck_to_' + j[:3] + j[-1] + '_points_avg'])
        deviation_columns.append('neck_to_' + j[:3] + j[-1] + '_points_avg')
        
        Adult['body_center_to_' + j[:3] + j[-1] + '_points_avg_deviation'] = (Adult[
            'body_center_to_' + j[:3] + j[-1] + '_points_avg'].mean() - Adult[
            'body_center_to_' + j[:3] + j[-1] + '_points_avg'])
        deviation_columns.append('body_center_to_' + j[:3] + j[-1] + '_points_avg')
        
        Adult['tailbase_to_' + j[:3] + j[-1] + '_points_avg_deviation'] = (Adult[
            'tailbase_to_' + j[:3] + j[-1] + '_points_avg'].mean() - Adult[
            'tailbase_to_' + j[:3] + j[-1] + '_points_avg'])
        deviation_columns.append('tailbase_to_' + j[:3] + j[-1] + '_points_avg')
        
    # Deviations for adult-nest distances
    
    Adult['neck_to_nest_centroid_home_deviation'] = (Adult[
            'neck_to_nest_centroid_home'].mean() - Adult[
            'neck_to_nest_centroid_home'])
    deviation_columns.append('neck_to_nest_centroid_home')
    
    Adult['body_center_to_nest_centroid_home_deviation'] = (Adult[
            'body_center_to_nest_centroid_home'].mean() - Adult[
            'body_center_to_nest_centroid_home'])
    deviation_columns.append('body_center_to_nest_centroid_home')
    
    Adult['tailbase_to_nest_centroid_home_deviation'] = (Adult[
            'tailbase_to_nest_centroid_home'].mean() - Adult[
            'tailbase_to_nest_centroid_home'])
    deviation_columns.append('tailbase_to_nest_centroid_home')
    
    # Deviations for infant-infant distances
    
    for j in infants_list:
        for k in infants_list:
            if (j[:3] + j[-1] + '_points_avg_to_' + k[:3] + k[-1] + '_points_avg_deviation') in Adult.columns:
                Adult[j[:3] + j[-1] + '_points_avg_to_' + k[:3] + k[-1] + '_points_avg'] = (Adult[
                    j[:3] + j[-1] + '_points_avg_to_' + k[:3] + k[-1] + '_points_avg'].mean() - Adult[
                    j[:3] + j[-1] + '_points_avg_to_' + k[:3] + k[-1] + '_points_avg'])
                deviation_columns.append(j[:3] + j[-1] + '_points_avg_to_' + k[:3] + k[-1] + '_points_avg')
                
    # Deviations for angles
    
    Adult['adult_angles_deviation'] = (Adult[
        'adult_angles'].mean() - Adult['adult_angles'])
    deviation_columns.append('adult_angles')
    
    for i in infants_list:
        Adult['adult_infant' + i[-1] + '_angle_deviation'] = (Adult[
              'adult_infant' + i[-1] + '_angle'].mean() - Adult['adult_infant' + i[-1] + '_angle'])
        deviation_columns.append('adult_infant' + i[-1] + '_angle')
        
    Adult['adult_nest_home_angle_deviation'] = (Adult[
        'adult_nest_home_angle'].mean() - Adult['adult_nest_home_angle'])
    deviation_columns.append('adult_nest_home_angle')
    
    # Rolling windows for the deviations features above
    
    for j in deviation_columns:
        for i in range(len(roll_windows_values)):
            currentColName = j + '_mean_' + str(roll_windows_values[i])
            currentDev_colName = currentColName + '_deviation'
            Adult[currentDev_colName] = (Adult[currentColName].mean() - Adult[currentColName])
    
########## PERCENTILE RANKS ##########


    # Use most of the same columns used in deviations for percentile calculations (except for area and angle)
    
    for j in deviation_columns:
        if not 'angle' in j:
            Adult[j + '_percentile_rank'] = Adult[j].rank(pct=True)
            
########## CALCULATE STRAIGHTNESS OF POLYLINE PATH: tortuosity ##########

    as_strided = np.lib.stride_tricks.as_strided
    win_size = 3
    centroidList_Adult_x = as_strided(Adult.body_center_x, (len(Adult) - (win_size - 1), win_size),
                                      (Adult.body_center_x.values.strides * 2))
    centroidList_Adult_y = as_strided(Adult.body_center_y, (len(Adult) - (win_size - 1), win_size),
                                      (Adult.body_center_y.values.strides * 2))
    
    for k in range(len(roll_windows_values)):
        start = 0
        end = start + int(roll_windows_values[k])
        tortuosity_Adult = []
        for y in range(len(Adult)):
            tortuosity_List_Adult = []
            CurrCentroidList_Adult_x = centroidList_Adult_x[start:end]
            CurrCentroidList_Adult_y = centroidList_Adult_y[start:end]
            
            for i in range(len(CurrCentroidList_Adult_x)):
                currMovementAngle_Adult = (
                    angle3pt(CurrCentroidList_Adult_x[i][0], CurrCentroidList_Adult_y[i][0],
                             CurrCentroidList_Adult_x[i][1], CurrCentroidList_Adult_y[i][1],
                             CurrCentroidList_Adult_x[i][2], CurrCentroidList_Adult_y[i][2]))
                tortuosity_List_Adult.append(currMovementAngle_Adult)
            tortuosity_Adult.append(sum(tortuosity_List_Adult) / (2 * math.pi))
            start += 1
            end += 1
        currentColName = str('Tortuosity_Adult_') + str(roll_windows_values[k])
        Adult[currentColName] = tortuosity_Adult

########## CALC THE NUMBER OF LOW PROBABILITY DETECTIONS & TOTAL PROBABILITY VALUE FOR ROW ##########
    
    # Make list of number of infants
    colsPerInfant = 12
    infants_num = int(len(Raw_infants.columns) / colsPerInfant)
    infants_list = []
    for i in range(infants_num):
        infants_list.append('Infant' + str(i + 1))
    
    # Reassign the column names as with the adult column names
    columns_name = []
    for column in Raw_infants.columns:
            
        if "likelihood" in column:
            columns_name.append(column[1] + '_' + column[2] + '_p')
        else:
            columns_name.append(column[1] + '_' + column[2] + '_' + column[3])
        
    Raw_infants.columns = columns_name
    
    
    # Low prob detections for adult (sum of prob columns, deviations and percentiles)
    adult_probability_columns = []
    
    Adult['sum_probabilities_adult'] = pd.Series(np.zeros(len(Adult)))
    for column in adult_original.columns:
        if '_p' in column:
            adult_probability_columns.append(column)
            Adult['sum_probabilities_adult'] += adult_original[column]
    Adult['sum_probabilities_adult_deviation'] = (Adult['sum_probabilities_adult'].mean() - 
                                        Adult['sum_probabilities_adult'])
    Adult['sum_probabilities_adult_deviation_percentile_rank'] = Adult[
          'sum_probabilities_adult_deviation'].rank(pct=True)
    Adult['sum_probabilities_adult_percentile_rank'] = Adult[
           'sum_probabilities_adult'].rank(pct=True)
    
    Adult_probability = adult_original.filter(adult_probability_columns)
    
    # For each frame, number of adult probability columns below a given threshold
    
    values_in_range_min, values_in_range_max = 0.0, 0.1
    Adult['Low_prob_detections_adult_0.1'] = Adult_probability.apply(
        func=lambda row: count_values_in_range(row, values_in_range_min, values_in_range_max), axis=1)
    values_in_range_min, values_in_range_max = 0.000000000, 0.5
    Adult['Low_prob_detections_adult_0.5'] = Adult_probability.apply(
        func=lambda row: count_values_in_range(row, values_in_range_min, values_in_range_max), axis=1)
    values_in_range_min, values_in_range_max = 0.000000000, 0.75
    Adult['Low_prob_detections_adult_0.75'] = Adult_probability.apply(
        func=lambda row: count_values_in_range(row, values_in_range_min, values_in_range_max), axis=1)
    
    # Low prob detections for infants (sum of prob columns, deviations and percentiles)
    Adult['sum_probabilities_infants'] = pd.Series(np.zeros(len(Adult)))
    infant_probability_columns = []
    for column in Raw_infants.columns:
        if '_p' in column:
            infant_probability_columns.append(column)
            Adult['sum_probabilities_infants'] += Raw_infants[column]
    Adult['sum_probabilities_infants_deviation'] = (Adult['sum_probabilities_infants'].mean() - 
                                        Adult['sum_probabilities_infants'])
    Adult['sum_probabilities_infants_deviation_percentile_rank'] = Adult[
          'sum_probabilities_infants_deviation'].rank(pct=True)
    Adult['sum_probabilities_infants_percentile_rank'] = Adult[
           'sum_probabilities_infants'].rank(pct=True)
    
    Infants_probability = Raw_infants.filter(infant_probability_columns)
    
    # For each frame, number of adult probability columns below a given threshold
    values_in_range_min, values_in_range_max = 0.0, 0.1
    Adult['Low_prob_detections_infants_0.1'] = Infants_probability.apply(
        func=lambda row: count_values_in_range(row, values_in_range_min, values_in_range_max), axis=1)
    values_in_range_min, values_in_range_max = 0.000000000, 0.5
    Adult['Low_prob_detections_infants_0.5'] = Infants_probability.apply(
        func=lambda row: count_values_in_range(row, values_in_range_min, values_in_range_max), axis=1)
    values_in_range_min, values_in_range_max = 0.000000000, 0.75
    Adult['Low_prob_detections_infants_0.75'] = Infants_probability.apply(
        func=lambda row: count_values_in_range(row, values_in_range_min, values_in_range_max), axis=1)
    
    Adult = Adult.fillna(0)
    
    #Save output to csv
    csvFile = (x + "_feature_added" + "." +'csv')
    csvFile = os.path.join(output_directory, csvFile)
    Adult.to_csv(csvFile, index=False)

19.76479342
18.76479808
